Connect to sheets

In [1]:
import pandas as pd
import streamlit as st
from streamlit_gsheets import GSheetsConnection
from datetime import datetime

In [2]:
url = "https://docs.google.com/spreadsheets/d/1xVKoVmKZOpNZ1oBySdLTE5GXZwN66UNoKjaBoEifZss/edit?usp=sharing"

# Create a connection object
conn = st.connection("gsheets", type=GSheetsConnection)

# Create raw Budget dataframe
budget_df = conn.read(spreadsheet =url, usecols=list(range(15)), worksheet=832591380, ttl=5)

# Create raw Actual dataframe
actual_df = conn.read(spreadsheet =url, usecols=list(range(15)), worksheet=487806377, ttl=5)

2025-05-08 16:37:28.217 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 16:37:29.014 
  command:

    streamlit run /Users/oscaraguilarembila/miniconda3/envs/ML/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-08 16:37:29.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 16:37:29.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 16:37:29.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 16:37:29.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 16:37:29.488 No runtime found, using MemoryCacheStorageManager
2025-05-08 16:37:29.493 No runtime found, using MemoryCacheStorageMa

In [3]:
# Make a copy of both dfs
b_df = budget_df.copy()
a_df = actual_df.copy()

# Rename columns
b_df.columns = ["Type", "Buckets"] + list(budget_df.iloc[3])[2:] # Budget
a_df.columns = ["Type", "Buckets"] + list(actual_df.iloc[3])[2:] # Actual

# Remove unnecessary rows
b_df = b_df.iloc[26:48] # Budget
a_df = a_df.iloc[26:48] # Actual

# Store Type and Buckets array in a Dataframe
categories = pd.DataFrame({
	"Type": b_df.Type,
	"Bucket": b_df.Buckets
})

# Remove unnecessary columns
b_df.drop(b_df.columns[[0, -1]], axis=1, inplace=True) # Budget
a_df.drop(a_df.columns[[0, -1]], axis=1, inplace=True) # Actual

# Reset index for all dataframes
b_df = b_df.reset_index(drop=True).drop("Buckets", axis=1) # Budget
a_df = a_df.reset_index(drop=True).drop("Buckets", axis=1) # Actual
categories = categories.reset_index(drop=True)

In [4]:
# Remove comas from all columns
budget = b_df.applymap(lambda x: str(x).replace(',', ''))
for col in a_df.select_dtypes(include='object'):
    a_df[col] = a_df[col].str.replace(',', '', regex=False)

# Convert to numbers
budget = budget.apply(pd.to_numeric)
actuals = a_df.apply(pd.to_numeric)

# Fill NA with 0
budget = budget.fillna(0)
actuals = actuals.fillna(0)

# Get today's month
td = datetime.today().strftime("%b")

/var/folders/g8/7typdv_j24q8mrqfwd3816r00000gn/T/ipykernel_10464/2850424433.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  budget = b_df.applymap(lambda x: str(x).replace(',', ''))


In [5]:
final_df = pd.DataFrame({
    'Type': categories.Type,
    'Category': categories.Bucket,
    'Balance': budget[td] - actuals[td],
    'Budget': budget[td]
})
final_df

,Type,Category,Balance,Budget
0,N,Food,97.0,400
1,N,Supplies,50.0,50
2,N,Health Care,20.0,30
3,N,Self Care,20.0,20
4,N,Carro Maintenance,15.0,15
5,N,Ropa,15.0,15
6,W,Escuela,381.0,500
7,W,Isaac,20.0,20
8,W,Adri,0.0,30
9,W,Eliam,10.0,10


In [7]:
final_df.Category.to_list()

['Food',
 'Supplies',
 'Health Care',
 'Self Care',
 'Carro Maintenance',
 'Ropa',
 'Escuela',
 'Isaac',
 'Adri',
 'Eliam',
 'Dates/Fun',
 'Subscriptions',
 'Gifts',
 'Fast Food',
 'Junk Food',
 'Viajes ',
 'Emergency Fond',
 'Debt/Fees',
 'Savings',
 'Investing',
 'Negocio',
 'Miscelaneous']

In [ ]:
col_inter = ['Food', 'Supplies', 'Self Care', 'Isaac', 'Adri', 'Eliam', 'Dates/Fun', 'Gifts', 'Dineout', 'Snakcs']

filtered_df = final_df[final_df['Category'].isin(col_inter)]

In [13]:
filtered_df

,Type,Category,Balance,Budget
0,N,Food,225.0,400
1,N,House Supplies,50.0,50
3,N,Self Care,20.0,20
7,W,Isaac,20.0,20
8,W,Adri,30.0,30
9,W,Eliam,10.0,10
10,W,Dates/Fun,50.0,50
12,W,Gifts,40.0,40
13,W,Fast Food,30.0,30
14,W,Junk Food,15.0,15


In [17]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(go.Indicator(
    type="indicator",
    mode="gauge+number+delta", # other modes for different display options
    value=50,
    title="Value on Bar",
    gauge={
        'shape': 'bullet',
        'bar': {
            'thickness': 0.8,
            'color': 'blue'  # Example color
        },
        'axis': {
            'visible': True,
            'range': [0, 100]
        }
    }
))
fig.show()

In [ ]:
# Save to pickle
final_df.to_pickle("April")

In [21]:
# Group by Type
final_df.groupby("Type").agg({'Balance':'sum', 'Budget':'sum'}).reset_index()

,Type,Balance,Budget
0,N,355.0,530
1,SD,1022.0,1300
2,W,785.0,785


In [22]:
# Filter Categories of interest
final_df[final_df['Balance'] > 0]

,Type,Category,Balance,Budget
0,N,Food,225.0,400
1,N,House Supplies,50.0,50
2,N,Health Care,30.0,30
3,N,Self Care,20.0,20
4,N,Carro Maintenance,15.0,15
5,N,Ropa,15.0,15
6,W,Escuela,500.0,500
7,W,Isaac,20.0,20
8,W,Adri,30.0,30
9,W,Eliam,10.0,10


### Next Steps
- Plotly display
- Diplay in Streamlit
- Make it look nice
- Add tab to insert expenses

https://plotly.com/python/bullet-charts/
https://plotly.com/python/indicator/

In [ ]:
col_inter = ['Food', 'House Supplies', 'Self Care', 'Isaac', 'Adri', 'Eliam', 'Dates/Fun', 'Gifts', 'Fast Food', 'Junk Food']

In [ ]:
sheet_id = '1xVKoVmKZOpNZ1oBySdLTE5GXZwN66UNoKjaBoEifZss'

In [5]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import streamlit as st
import pandas as pd

# Authenticate and connect to Google Sheets
def connect_to_gsheet(creds_json, spreadsheet_name, sheet_name):
    scope = ["https://spreadsheets.google.com/feeds", 
             'https://www.googleapis.com/auth/spreadsheets',
             "https://www.googleapis.com/auth/drive.file", 
             "https://www.googleapis.com/auth/drive"]
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(creds_json, scope)
    client = gspread.authorize(credentials)
    spreadsheet = client.open(spreadsheet_name)  
    return spreadsheet.worksheet(sheet_name)  # Access specific sheet by name

# Google Sheet credentials and details
SPREADSHEET_NAME = 'Cash Flow Fam Aguilar'
SHEET_NAME = 'Real'
CREDENTIALS_FILE = './credentials.json'

# Connect to the Google Sheet
sheet_by_name = connect_to_gsheet(CREDENTIALS_FILE, SPREADSHEET_NAME, sheet_name=SHEET_NAME)

In [ ]:
# Get the values of row 5 and column B
row_5 = sheet_by_name.row_values(5)
col_B = sheet_by_name.col_values(2)

# Find the column and row indexes
col_index = row_5.index("May") + 1
row_index = col_B.index("Food") + 1

# Read current value
current_value = sheet_by_name.cell(row_index, col_index).value
new_value = float(current_value or 0) + 10

# Update Category
sheet_by_name.update_cell(row_index, col_index, new_value)

# Update Debt
sheet_by_name.update_cell(row_index, col_index, new_value)
row_index = col_B.index("Deuda") + 1
sheet_by_name.update_cell(row_index, col_index, float(sheet_by_name.cell(row_index, col_index).value)+10)

# Update Card
row_index = col_B.index(card) + 1
sheet_by_name.update_cell(row_index, col_index, float(sheet_by_name.cell(row_index, col_index).value)+10)

' 175 '

In [7]:
new_value = float(current_value or 0) + 10
new_value


185.0

In [ ]:

# Write the updated value
sheet_by_name.update_cell(row_index, col_index, new_value)
